##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# DTensor による分散型トレーニング


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/dtensor_ml_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/dtensor_ml_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/dtensor_ml_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/distribute/dtensor_ml_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 概要

DTensor を使用すると、デバイス間でモデルのトレーニングを分散し、有効性、信頼性、およびスケーラビリティを改善することができます。DTensor の概念についての詳細は、[DTensor プログラミングガイド](https://www.tensorflow.org/guide/dtensor_overview)をご覧ください。

このチュートリアルでは、DTensor を使って、センチメント分析モデルをトレーニングします。この例では、以下の 3 つの分散型トレーニングスキームについて紹介します。

- データ並列トレーニング: トレーニングサンプルを複数のデバイスにシャーディング（分割）します。
- モデル並列トレーニング: モデル変数を複数のデバイスにシャーディングします。
- 空間並列トレーニング: 入力データの特徴量を複数のデバイスにシャーディングします（[空間分割](https://cloud.google.com/blog/products/ai-machine-learning/train-ml-models-on-large-images-and-3d-volumes-with-spatial-partitioning-on-cloud-tpus)としても知られています）。

このチュートリアルのトレーニングの部分は、[センチメント分析に関する Kaggle ガイド](https://www.kaggle.com/code/anasofiauzsoy/yelp-review-sentiment-analysis-tensorflow-tfds/notebook)ノートブックを基盤としています。完全なトレーニングと評価のワークフロー（DTensor なし）について学習するには、そちらのノートブックをご覧ください。

このチュートリアルでは、以下のステップを説明します。

- まず、データクリーニングを行い、トークン化された文とその極性の `tf.data.Dataset` を取得します。

- 次に、カスタム Dense レイヤーと BatchNorm レイヤーを使って MLP モデルを構築します。推論変数の追跡には、`tf.Module` を使用します。モデルコンストラクタは、追加の `Layout` 引数を取って、変数のシャーディングを制御します。

- トレーニングには、はじめに `tf.experimental.dtensor` のチェックポイント機能を使ってデータ並列トレーニングを使用します。次に、モデル並列トレーニングと空間並列トレーニングを使用します。

- 最後のセクションでは、TensorFlow 2.9 時点での `tf.saved_model` と `tf.experimental.dtensor` の対話を簡単に説明します。


## MNIST モデルをビルドする

DTensor は、TensorFlow 2.9.0 リリースに含まれています。

In [ ]:
!pip install --quiet --upgrade --pre tensorflow tensorflow-datasets

次に、`tensorflow` と `tensorflow.experimental.dtensor` をインポートし、8 個の仮想 CPU を使用するように TensorFlow を構成します。

この例では CPU を使用しますが、DTensor は CPU、GPU、または TPU デバイスで同じように動作します。

In [ ]:
import tempfile
import numpy as np
import tensorflow_datasets as tfds

import tensorflow as tf

from tensorflow.experimental import dtensor
print('TensorFlow version:', tf.__version__)

In [ ]:
def configure_virtual_cpus(ncpu):
  phy_devices = tf.config.list_physical_devices('CPU')
  tf.config.set_logical_device_configuration(phy_devices[0], [
        tf.config.LogicalDeviceConfiguration(),
    ] * ncpu)

configure_virtual_cpus(8)
DEVICES = [f'CPU:{i}' for i in range(8)]

tf.config.list_logical_devices('CPU')

## データセットをダウンロードする

センチメント分析モデルをトレーニングするための IMDB レビューデータセットをダウンロードします。

In [ ]:
train_data = tfds.load('imdb_reviews', split='train', shuffle_files=True, batch_size=64)
train_data

## データを準備する

まず、テキストをトークン化します。ここでは、One-Hot エンコーディングの拡張機能である `'tf_idf'` モードの `tf.keras.layers.TextVectorization` を使用します。

- 速度を得るために、トークン数は 1200 に制限します。
- `tf.Module` を単純に維持するために、トレーニングの前のプリプロセッシングステップとして `TextVectorization` を実行します。

データクリーニングセクションの最終結果は、トークン化したテキストを `x`、ラベルを `y` とした `Dataset` です。

**注意**: プリプロセッシングステップとして `TextVectorization` を実行するのは、**通常の実践でも推奨される実践もありません**。こうすることで、トレーニングデータがクライアントメモリに収まることが想定されますが、常にそうであるとは限りません。


In [ ]:
text_vectorization = tf.keras.layers.TextVectorization(output_mode='tf_idf', max_tokens=1200, output_sequence_length=None)
text_vectorization.adapt(data=train_data.map(lambda x: x['text']))

In [ ]:
def vectorize(features):
  return text_vectorization(features['text']), features['label']

train_data_vec = train_data.map(vectorize)
train_data_vec

## DTensor を使ってニューラルネットワークを構築する

では、DTensor を使って多層パーセプトロン（MLP）ネットワークを構築しましょう。このネットワークでは、全結合の Dense と BatchNorm レイヤーを使用します。

`DTensor` は、入力 `Tensor` と変数の `dtensor.Layout` 属性に従って、通常の TensorFlow Ops の単一プログラムマルチデータ（SPMD）拡張を通じて TensorFlow を拡張します。

`DTensor` を認識するレイヤーの変数は `dtensor.DVariable` で、`DTensor` を認識するレイヤーオブジェクトのコンストラクタは、通常のレイヤーパラメータの他に追加の `Layout` 入力を取ります。

注意: TensorFlow 2.9 の時点では、`tf.keras.layer.Dense` や `tf.keras.layer.BatchNormalization` などの Keras レイヤーは、`dtensor.Layout` 引数を受け取ります。DTensor を使って Keras を使用する方法の詳細については、[DTensor と Keras の統合チュートリアル](/tutorials/distribute/dtensor_keras_tutorial)をご覧ください。

### Dense レイヤー

以下のカスタム Dense レイヤーは、2 つのレイヤー変数を定義します。1 つは重みの変数 $W_{ij}$、もう 1 つはバイアスの変数 $b_i$ です。

$$ y_j = \sigma(\sum_i x_i W_{ij} + b_j) $$


### Layout の推論

この結果は、以下の観察結果から得られます。

- 行列内積 $t_j = \sum_i x_i W_{ij}$ のオペランドに推奨される DTensor シャーディングは、$i$ 軸に沿って $\mathbf{W}$ と $\mathbf{x}$ を同じ方法でシャーディングすることです。

- 行列和 $t_j + b_j$ のオペランドに推奨される DTensor シャーディングは、$j$ 軸に沿って $\mathbf{t}$ と $\mathbf{b}$ を同じ方法でシャーディングすることです。


In [ ]:
class Dense(tf.Module):

  def __init__(self, input_size, output_size,
               init_seed, weight_layout, activation=None):
    super().__init__()

    random_normal_initializer = tf.function(tf.random.stateless_normal)

    self.weight = dtensor.DVariable(
        dtensor.call_with_layout(
            random_normal_initializer, weight_layout,
            shape=[input_size, output_size],
            seed=init_seed
            ))
    if activation is None:
      activation = lambda x:x
    self.activation = activation
    
    # bias is sharded the same way as the last axis of weight.
    bias_layout = weight_layout.delete([0])

    self.bias = dtensor.DVariable(
        dtensor.call_with_layout(tf.zeros, bias_layout, [output_size]))

  def __call__(self, x):
    y = tf.matmul(x, self.weight) + self.bias
    y = self.activation(y)

    return y

### BatchNorm

バッチ正規化レイヤーでは、トレーニング中にモードが崩壊するのを回避できます。この場合は、バッチ正則化レイヤーを追加することで、モデルのトレーニングでゼロのみを生成するモデルが生成されないようにすることができます。

以下のカスタム `BatchNorm` レイヤーのコンストラクタは、`Layout` 引数を取りません。これは、`BatchNorm` にレイヤー変数がないためです。ただし、レイヤーへの唯一の入力である 'x' がすでにグローバルバッチを表現する DTensor であるため、DTensor では機能します。

注意: DTensor では、入力 Tensor 'x' は常にグローバルバッチを表現します。したがって、`tf.nn.batch_normalization` はグローバルバッチに適用されます。これは、Tensor 'x'  がバッチのレプリカ単位のシャード（ローカルバッチ）のみを表現する `tf.distribute.MirroredStrategy` を使ってトレーニングとは異なります。

In [ ]:
class BatchNorm(tf.Module):

  def __init__(self):
    super().__init__()

  def __call__(self, x, training=True):
    if not training:
      # This branch is not used in the Tutorial.
      pass
    mean, variance = tf.nn.moments(x, axes=[0])
    return tf.nn.batch_normalization(x, mean, variance, 0.0, 1.0, 1e-5)

フル機能のバッチ正規化レイヤー（`tf.keras.layers.BatchNormalization` など）は、変数に Layout 引数が必要となります。

In [ ]:
def make_keras_bn(bn_layout):
  return tf.keras.layers.BatchNormalization(gamma_layout=bn_layout,
                                            beta_layout=bn_layout,
                                            moving_mean_layout=bn_layout,
                                            moving_variance_layout=bn_layout,
                                            fused=False)

### すべてのレイヤーをまとめる

次に、上記のビルディングブロックを使って、多層パーセプトロン（MLP）ネットワークを構築しましょう。下の図では、DTensor シャーディングまたは複製を適用しない 2 つの `Dense` レイヤーの入力 `x` と重み行列を示します。

<img src="https://www.tensorflow.org/images/dtensor/no_dtensor.png" class="" alt="非分散型モデルの入力と重み行列。">


最初の `Dense` レイヤーの出力は、2 つ目の `Dense` レイヤーの入力に渡されます（`BatchNorm` の後）。したがって、最初の `Dense` レイヤー（$\mathbf{W_1}$）の出力と 2 つ目の `Dense` レイヤー（$\mathbf{W_2}$）の入力に推奨される DTensor シャーディングは、$\mathbf{W_1}$ と $\mathbf{W_2}$ を共通する軸 $\hat{j}$ に沿って同じ方法でシャーディングすることです。

$$ \mathsf{Layout}[{W_{1,ij}}; i, j] = \left[\hat{i}, \hat{j}\right] \ \mathsf{Layout}[{W_{2,jk}}; j, k] = \left[\hat{j}, \hat{k} \right] $$

レイアウトの推論では、2 つのレイアウトが独立していないことが示されていますが、モデルインターフェイスを単純化するために、`MLP` は Dense レイヤーごとに1つずつ、2 つの `Layout` 引数を取ります。

In [ ]:
from typing import Tuple

class MLP(tf.Module):

  def __init__(self, dense_layouts: Tuple[dtensor.Layout, dtensor.Layout]):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dense_layouts[0], activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dense_layouts[1])

  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y


レイアウト推論の制約の正確さと API の単純さの間に発生するトレードオフは、DTensor を使用する API の一般的な設計ポイントです。別の API を使用して `Layout` 間の依存関係をキャプチャすることも可能です。たとえば、`MLPStricter` クラスはコンストラクタに `Layout` オブジェクトを作成します。

In [ ]:
class MLPStricter(tf.Module):

  def __init__(self, mesh, input_mesh_dim, inner_mesh_dim1, output_mesh_dim):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dtensor.Layout([input_mesh_dim, inner_mesh_dim1], mesh),
        activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dtensor.Layout([inner_mesh_dim1, output_mesh_dim], mesh))


  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y

モデルが確実に実行するように、完全に複製されたレイアウトと完全に複製された `'x'` 入力のバッチを使用してモデルをプローブします。

In [ ]:
WORLD = dtensor.create_mesh([("world", 8)], devices=DEVICES)

model = MLP([dtensor.Layout.replicated(WORLD, rank=2),
             dtensor.Layout.replicated(WORLD, rank=2)])

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x = dtensor.copy_to_mesh(sample_x, dtensor.Layout.replicated(WORLD, rank=2))
print(model(sample_x))

## データをデバイスに移動する

通常、`tf.data` イテレータ（およびその他のデータの取得手法）によって、ローカルホストのデバイスメモリにバックアップされるテンソルオブジェクトが生成されます。このデータは、DTensor のコンポーネントテンソルをバックアップするアクセラレータデバイスのメモリに転送する必要があります。

このような状況においては、`dtensor.copy_to_mesh` は適していません。DTensor はグローバル観点であるため、すべてのデバイスに入力テンソルを複製してしまうためです。そのため、このチュートリアルでは、データの転送を容易にするヘルパー関数 `repack_local_tensor` を使用します。このヘルパー関数は、レプリカをバックアップするデバイスに、グローバルバッチのレプリカ用のシャードを送信する（送信するだけです）`dtensor.pack` を使用します。

単純化されたこの関数は、シングルクライアントを想定しています。マルチクライアントアプリケーションでは、ローカルテンソルを分割する正しい方法と、Split とローカルデバイスのマッピングを特定するには、多大な労力が必要となる可能性があります。

`tf.data` の統合を単純化する追加の DTensor API が計画されており、シングルクライアントとマルチクライアントの両方のアプリケーションがサポートされる予定です。ご期待ください。

In [ ]:
def repack_local_tensor(x, layout):
  """Repacks a local Tensor-like to a DTensor with layout.

  This function assumes a single-client application.
  """
  x = tf.convert_to_tensor(x)
  sharded_dims = []

  # For every sharded dimension, use tf.split to split the along the dimension.
  # The result is a nested list of split-tensors in queue[0].
  queue = [x]
  for axis, dim in enumerate(layout.sharding_specs):
    if dim == dtensor.UNSHARDED:
      continue
    num_splits = layout.shape[axis]
    queue = tf.nest.map_structure(lambda x: tf.split(x, num_splits, axis=axis), queue)
    sharded_dims.append(dim)

  # Now we can build the list of component tensors by looking up the location in
  # the nested list of split-tensors created in queue[0].
  components = []
  for locations in layout.mesh.local_device_locations():
    t = queue[0]
    for dim in sharded_dims:
      split_index = locations[dim]  # Only valid on single-client mesh.
      t = t[split_index]
    components.append(t)

  return dtensor.pack(components, layout)

## データ並列トレーニング

このセクションでは、データ並列トレーニング使用して、MLP モデルをトレーニングします。その後のセクションでは、モデル並列トレーニングと空間並列トレーニングについて説明します。

データ並列トレーニングは、分散型機械学習で一般的に使用されているスキームです。

- モデル変数は、N 個のデバイスにそれぞれ複製されます。
- グローバルバッチは、N 個のレプリカごとのバッチに分割されます。
- それぞれのレプリカごとのバッチは、レプリカデバイスでトレーニングされます。
- 勾配は、すべてのレプリカでデータの重み付けが集団的に実行される前に減らされます。

データ並列トレーニングでは、デバイスの数に関してほぼ直線的なスピードアップが得られます。

### データ並列メッシュを作成する

典型的なデータ並行トレーニングループは、単一の `batch` 次元で構成される DTensor `Mesh` を使用します。この場合、各デバイスは、グローバルバッチからシャードを受け取るモデルのレプリカとなります。

<img src="https://www.tensorflow.org/images/dtensor/dtensor_data_para.png" class="" alt="データ並列メッシュ">

複製されたモデルはレプリカで実行するため、モデル変数が完全に複製されます（シャーディングされません）。

In [ ]:
mesh = dtensor.create_mesh([("batch", 8)], devices=DEVICES)

model = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),
             dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),])


### トレーニングデータを DTensor にパッキングする

トレーニングデータバッチは、DTensor がトレーニングデータを `'batch'` メッシュ次元に均等に分散するように、`'batch'`(first) 軸に沿ってシャーディングされて DTensor にパックされます。

**注意**: DTensor では、`batch size` は常にグローバルバッチサイズを指します。バッチサイズは、`batch` メッシュ次元のサイズで均等に分割されるように選択します。

In [ ]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x, sample_y = repack_batch(sample_x, sample_y, mesh)

print('x', sample_x[:, 0])
print('y', sample_y)

### トレーニングステップ

この例では、カスタムトレーニングループ（CTL）で確率的勾配降下法オプティマイザを使用します。このトピックについての詳細は、[カスタムトレーニングループガイド](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)と[ウォークスルー](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough)をご覧ください。

`train_step` は、この本体が TensorFlow Graph としてトレースされることを示すために、`tf.function` としてカプセル化されます。`train_step` の本体は、前方推論パス、後方勾配パス、および変数更新で構成されています。

`train_step` の本体には特殊な DTensor アノテーションが含まれないことに注意してください。代わりに、`train_step` には、入力バッチとモデルのグローバルビューから入力 `x` と `y` を処理する高レベルの TensorFlow 演算子のみが含まれています。すべての DTensor アノテーション（`Mesh`, `Layout`）は、トレーニングステップから除外されます。

In [ ]:
# Refer to the CTL (custom training loop guide)
@tf.function
def train_step(model, x, y, learning_rate=tf.constant(1e-4)):
  with tf.GradientTape() as tape:
    logits = model(x)
    # tf.reduce_sum sums the batch sharded per-example loss to a replicated
    # global loss (scalar).
    loss = tf.reduce_sum(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=y))
  parameters = model.trainable_variables
  gradients = tape.gradient(loss, parameters)
  for parameter, parameter_gradient in zip(parameters, gradients):
    parameter.assign_sub(learning_rate * parameter_gradient)

  # Define some metrics
  accuracy = 1.0 - tf.reduce_sum(tf.cast(tf.argmax(logits, axis=-1, output_type=tf.int64) != y, tf.float32)) / x.shape[0]
  loss_per_sample = loss / len(x)
  return {'loss': loss_per_sample, 'accuracy': accuracy}

### チェックポイントを設定する

`dtensor.DTensorCheckpoint` を使用して、DTensor モデルにチェックポイントを設定できます。DTensor チェックポイントのフォーマットは標準の TensorFlow チェックポイントに完全に対応しています。`dtensor.DTensorCheckpoint` を `tf.train.Checkpoint` に統合する作業が進められています。

DTensor チェックポイントがリストアされると、変数の `Layout` がチェックポイントが作成されたときと異なる場合があります。このチュートリアルではこの特徴を活用して、モデル並列トレーニングと空間並列トレーニングのセクションに続けています。


In [ ]:
CHECKPOINT_DIR = tempfile.mkdtemp()

def start_checkpoint_manager(mesh, model):
  ckpt = dtensor.DTensorCheckpoint(mesh, root=model)
  manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_DIR, max_to_keep=3)

  if manager.latest_checkpoint:
    print("Restoring a checkpoint")
    ckpt.restore(manager.latest_checkpoint).assert_consumed()
  else:
    print("new training")
  return manager


### トレーニングループ

データ並列トレーニングスキームの場合、トレーニングを数エポック行って、その進捗をレポートします。モデルのトレーニングには 3 エポックでは不十分です。精度 50% は、適当な推定と同等と言えます。

後でトレーニングを再開できるように、チェックポイント設定を有効にします。以降のセクションにおいて、チェックポイントを読み込み、別の並列スキームでトレーニングを行います。

In [ ]:
num_epochs = 2
manager = start_checkpoint_manager(mesh, model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()), stateful_metrics=[])
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:

    x, y = repack_batch(x, y, mesh)

    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## モデル並列トレーニング

2 次元 `Mesh` に切り替えて、2 つ目のメッシュ次元に沿ってモデル変数をシャーディングすると、トレーニングがモデル並列になります。

モデル並列トレーニングでは、モデルの各レプリカは複数のデバイス（この場合は 2 つ）にまたがっています。

- 4 個のモデルレプリカがあり、トレーニングデータバッチは、その 4 個のレプリカに分散されます。
- 単一のモデルレプリカ内の 2 つのデバイスは、複製されたトレーニングデータを受け取ります。

<img src="https://www.tensorflow.org/images/dtensor/dtensor_model_para.png" alt="Model parallel mesh" class="">


In [ ]:
mesh = dtensor.create_mesh([("batch", 4), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout([dtensor.UNSHARDED, "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])

トレーニングデータは、バッチ次元に沿ってシャーディングされたままであるため、データ並列トレーニングの場合と同じ `repack_batch` 関数を再利用できます。DTensor は `"model"` メッシュ次元に沿って、レプリカごとのバッチをレプリカ内のすべてのデバイスに自動的に複製します。

In [ ]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

次に、トレーニングループを実行します。トレーニングループは、データ並列トレーニングの例と同じチェックポイントマネージャーを再利用するため、コードは全く同じです。

モデル並列トレーニングで、データ並列でトレーニングされたモデルのトレーニングを続けることができます。

In [ ]:
num_epochs = 2
manager = start_checkpoint_manager(mesh, model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:
    x, y = repack_batch(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))
    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## 空間並列トレーニング

非常に高次元のデータ（非常に大きな画像や動画など）をトレーニングする際は、特徴量次元に沿ってシャーディングすることが推奨される可能性があります。これは[空間分割](https://cloud.google.com/blog/products/ai-machine-learning/train-ml-models-on-large-images-and-3d-volumes-with-spatial-partitioning-on-cloud-tpus)と呼ばれる手法で、はじめは大きな 3D 入力サンプルでモデルをトレーニングするために TensorFlow に導入された手法です。

<img src="https://www.tensorflow.org/images/dtensor/dtensor_spatial_para.png" class="no-filter" alt="空間並列メッシュ">

DTensor はこのようなケースもサポートしています。唯一変更が必要なのは、`feature` 次元を含めて対応する `Layout` を適用するメッシュを作成することです。


In [ ]:
mesh = dtensor.create_mesh([("batch", 2), ("feature", 2), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout(["feature", "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])


入力テンソルを DTensor にパッキングする際に、`feature` 次元に沿って入力データをシャーディングします。この作業は、`repack_batch_for_spt` というわずかに異なる再パック関数を使って行います。ここで、`spt` は、空間並列トレーニング（Spatial Parallel Training）略です。

In [ ]:
def repack_batch_for_spt(x, y, mesh):
    # Shard data on feature dimension, too
    x = repack_local_tensor(x, layout=dtensor.Layout(["batch", 'feature'], mesh))
    y = repack_local_tensor(y, layout=dtensor.Layout(["batch"], mesh))
    return x, y

空間並列トレーニングも、他の並列トレーニングスキームで作成されたチェックポイントから続行することができます。

In [ ]:
num_epochs = 2

manager = start_checkpoint_manager(mesh, model)
for epoch in range(num_epochs):
  step = 0
  metrics = {'epoch': epoch}
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))

  for x, y in train_data_vec:
    x, y = repack_batch_for_spt(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## SavedModel と DTensor

DTensor と SavedModel の統合は、現在も開発中です。このセクションでは、TensorFlow 2.9.0 の現在の状況についてのみ説明します。

TensorFlow 2.9.0 の時点で、`tf.saved_model` は、完全に複製された変数を持つ DTensor モデルのみを受け入れます。

回避策としてチェックポイントを再読み込みすれば、DTensor モデルを完全に複製されたモデルに変換することが可能ではありますが、モデルが保存されると、すべての DTensor アノテーションが失われ、保存したシグネチャは DTensor ではなく通常の Tensor とのみ使用できるようになってしまいます。

In [ ]:
mesh = dtensor.create_mesh([("world", 1)], devices=DEVICES[:1])
mlp = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh), 
           dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)])

manager = start_checkpoint_manager(mesh, mlp)

model_for_saving = tf.keras.Sequential([
  text_vectorization,
  mlp
])

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def run(inputs):
  return {'result': model_for_saving(inputs)}

tf.saved_model.save(
    model_for_saving, "/tmp/saved_model",
    signatures=run)

TensorFlow 2.9.0 の時点では、読み込まれたシグネチャは通常の Tensor か完全に複製された DTensor（通常の Tensor に変換されます）を使ってのみ呼び出せます。

In [ ]:
sample_batch = train_data.take(1).get_single_element()
sample_batch

In [ ]:
loaded = tf.saved_model.load("/tmp/saved_model")

run_sig = loaded.signatures["serving_default"]
result = run_sig(sample_batch['text'])['result']

In [ ]:
np.mean(tf.argmax(result, axis=-1) == sample_batch['label'])

## 次のステップ

このチュートリアルでは、DTensor を使って MLP センチメント分析モデルの構築とトレーニングを行う方法を説明しました。

`Mesh` と `Layout` はプリミティブではありますが、DTensor は TensorFlow `tf.function` を、さまざまなトレーニングスキームに適した分散型プログラムに変換することができます。

実際の機械学習アプリケーションでは、評価とクロス検証を適用して、過学習モデルが生成されないようにする必要があります。このチュートリアルで紹介された手法を適用して、評価に並列性を導入することも可能です。

`tf.Module` を使ってモデルをゼロから構築するには多大な労力が必要であり、レイヤーやヘルパー関数と言った既存のビルディングブロックを再利用することで、モデル開発を大幅に高速化することができます。TensorFlow 2.9.0 の時点では、`tf.keras.layers` 以下のすべての Keras レイヤーは、その引数として DTensor レイアウトを受け入れ、DTensor モデルを構築するために使用することができます。また、モデルの実装を変更することなく、DTensor を使って直接 Keras モデルを再利用することも可能です。DTensor Keras の使用に関する詳細は、[DTensor と Keras の統合チュートリアル](https://www.tensorflow.org/tutorials/distribute/dtensor_keras_tutorial)をご覧ください。 